# Simple Example Using the Clingo Module

In [1]:
import cffi
import clingo
from clingo.control import Control

def run_clingo(args, files):
    print(f"clingo version {clingo.__version__}")
    ctl = Control(args, logger=lambda c, m: print(m))
   
    n = 0
    def on_model(m):
        nonlocal n
        n += 1
        print(f"Answer: {n}")
        print(" ".join(str(sym) for sym in m.symbols(shown=True)))

    print("Reading...")
    for file in files:
        ctl.load(file)
        
    print("Grounding...")
    ctl.ground()
    
    print("Solving...")
    ret = ctl.solve(on_model=on_model)
    if ret.satisfiable:
        print("SATISFIABLE")
    if ret.unsatisfiable:
        print("UNSATISFIABLE")
    
    summary = ctl.statistics['summary']
    print()
    print(f"Models       : {int(summary['models']['enumerated'])}")
    print(f"Calls        : {int(summary['call'] + 1)}")
    print(f"Time         : {summary['times']['total']:.3f}s")

In [5]:
with open("test.lp", "w") as fp:
    fp.write("""\
{a}.
a :- b.
{c; d}.
:- not 2 { a; b; c; d }.
""")

In [6]:
run_clingo(["0"], ["test.lp"])

clingo version 5.7.1
Reading...
Grounding...
test.lp:2:6-7: info: atom does not occur in any rule head:
  b

test.lp:4:15-16: info: atom does not occur in any rule head:
  b

Solving...
Answer: 1
c d
Answer: 2
a d
Answer: 3
a c d
Answer: 4
a c
SATISFIABLE

Models       : 4
Calls        : 1
Time         : 0.099s
